#### Thursday, December 7, 2023

[Intel Neural Chat 7B - Mistral meets new hardware & new data](https://www.youtube.com/watch?v=curhIJyuhYA)

https://colab.research.google.com/drive/14KvD_bMnXwLBqeKwaNZoWW4RqWt5kVBA?usp=sharing

In [ ]:
 !pip -q install git+https://github.com/huggingface/transformers # often need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain
!pip install -q -U git+https://github.com/huggingface/peft.git
# pip install -q -U git+https://github.com/huggingface/accelerate.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0

In [ ]:
!nvidia-smi

Tue Nov 28 23:26:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    43W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Intel/neural-chat-7b-v3-1



In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
torch.set_default_device('cuda')

In [ ]:


model = AutoModelForCausalLM.from_pretrained("Intel/neural-chat-7b-v3-1",
                                            #  load_in_4bit=True,

                                             )

tokenizer = AutoTokenizer.from_pretrained("Intel/neural-chat-7b-v3-1",
                                          )



config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

In [ ]:
!nvidia-smi

Tue Nov 28 23:27:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    49W / 400W |  29851MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"

# encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

# device = 'cuda'
# model_inputs = encodeds.to(device)
# # model.to(device)

# generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
# decoded = tokenizer.batch_decode(generated_ids)
# print(decoded[0])

In [ ]:
# pipe =

### Prompt Format
```
### System:
{system}
### User:
{usr}
### Assistant:

```

In [ ]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
# def generate(input_text, system_prompt="",max_length=512):
#     # prompt = f"""<s>[INST]{input_text}[/INST]"""
#     prompt = f"""{input_text}"""
#     inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
#     outputs = model.generate(**inputs,
#                              max_new_tokens=max_length,
#                              temperature=0.1,
#                              do_sample=True)
#     text = tokenizer.batch_decode(outputs)[0]
#     wrapped_text = wrap_text(text)
#     print(wrapped_text)

def generate(input_text, system_prompt="You are a helpful assistant",max_length=512):

    # Format the input using the provided template
    prompt = f"### System:\n{system_prompt}\n### User:\n{input_text}\n### Assistant:\n"

    # Tokenize and encode the prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False)

    # Generate a response
    outputs = model.generate(inputs,
                             max_new_tokens=max_length,
                             pad_token_id=2,
                             num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)


    # Extract only the assistant's response
    text = response.split("### Assistant:\n")[-1]
    wrapped_text = wrap_text(text)
    print(wrapped_text)


## Instruction Answering

In [ ]:
generate('Let me tell you an interesting story about cat Tom and mouse Jerry,',
         max_length=512)

 Once upon a time, in a cozy little house, there lived two very different creatures: a
mischievous cat named Tom and a clever mouse named Jerry. Tom was a lazy, yet cunning
feline who loved nothing more than a good nap. On the other hand, Jerry was a quick-witted
and resourceful mouse who was always on the lookout for food and ways to avoid Tom's
clutches.

One day, Jerry stumbled upon a delicious piece of cheese in Tom's kitchen. He knew he had
to act fast to secure this tasty treat for himself. As he carefully planned his escape, he
noticed Tom was sound asleep on the couch. Jerry saw his chance and quickly grabbed the
cheese, making a beeline for the door.

Tom, however, was not one to be easily fooled. He sensed something was amiss and quickly
sprang into action, chasing Jerry through the house. Jerry, being the clever mouse he was,
knew he had to use his wits to outsmart Tom. He darted through various rooms, hiding in
cupboards, and even climbing up the walls to avoid Tom's relent

In [ ]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         max_length=128)

 Mathematics, like a lighthouse, serves as a guiding beacon for understanding and
navigating the complexities of our world. Both are essential tools that provide clarity,
direction, and stability in their respective domains.

Mathematics, as a discipline, is a systematic study of numbers, shapes, patterns, and
relationships. It is a language that allows us to express and analyze various aspects of
the universe, from the smallest particles to the vastness of the cosmos. Like a
lighthouse, mathematics provides a stable foundation and a reliable reference point for us
to build upon.

A lighthouse


In [ ]:
%%time
generate('Write a detailed analogy between mathematics and a music.',
         max_length=256)

 Mathematics and music can be compared as two distinct yet interconnected artistic
disciplines that share several similarities in their structure, principles, and the way
they evoke emotions and stimulate the human mind.

1. Structure: Both mathematics and music have a well-defined structure that follows
specific rules and patterns. In mathematics, these rules are often represented by
formulas, equations, and theorems, while in music, they are expressed through notes,
scales, and chords. These structures provide a framework for creativity and innovation,
allowing for the creation of new ideas and concepts.

2. Harmony and balance: Both mathematics and music rely on the concept of harmony and
balance to create a sense of beauty and order. In mathematics, this is achieved through
the use of symmetries, proportions, and the golden ratio, which create a sense of balance
and aesthetic appeal. Similarly, in music, harmony is created through the combination of
different notes and chords, whic

In [ ]:
%%time
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         max_length=512)

 Llamas, Vicunas, and Alpacas are all South American camelids, which means they are
related species belonging to the Camelidae family. They share similarities in appearance
and behavior, but there are some key differences between them.

1. Llamas: Llamas are the largest of the three species, with an average height of around 6
feet and a weight of around 300 pounds. They have long necks, large ears, and a
distinctive humpless back. Llamas are primarily used as pack animals and for their wool,
which is coarser than that of alpacas.

2. Vicunas: Vicunas are the smallest of the three species, with an average height of
around 3.5 feet and a weight of around 100 pounds. They have a slender build, long necks,
and small ears. Vicunas are known for their soft, fine wool, which is highly valued for
its warmth and softness. They are also considered endangered due to overhunting for their
wool in the past.

3. Alpacas: Alpacas are medium-sized, with an average height of around 3.5 feet and a
weigh

In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         max_length=512)

 Subject: Open Sourcing GPT-4: A Valuable Contribution to the AI Community

Dear Sam,

I hope this email finds you well. As a strong advocate for the advancement of artificial
intelligence, I believe that OpenAI's GPT-4 has the potential to revolutionize the way we
interact with technology. With this in mind, I would like to share my thoughts on the
benefits of open sourcing GPT-4.

1. Collaboration and Innovation: By making GPT-4 open source, you would enable a global
community of developers, researchers, and enthusiasts to contribute to its development.
This collaboration would lead to a more robust and versatile model, ultimately benefiting
everyone.

2. Accessibility and Equality: Open sourcing GPT-4 would make it more accessible to a
wider range of users, including those who may not have the resources to access proprietary
AI models. This would help bridge the gap between those who can afford advanced technology
and those who cannot, promoting equality in the AI space.

3. Transpa

In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Freddy a young 5 year old boy who is scared AI will end the world, write only with the language of a young child!",
         max_length=512)

 Dear Sam Altman,

I am Freddy, a 5-year-old boy who loves learning new things. I heard about GPT-4, a very
smart computer that can do many tasks. I think it would be a great idea to share GPT-4
with everyone by making it open source. Here are some reasons why:

1. Sharing is Caring: When we share toys with our friends, we all have more fun. In the
same way, if everyone can use GPT-4, it will help people learn and create amazing things
together.

2. Making Friends: GPT-4 can be like a friend who helps us with our homework, teaches us
new skills, and even makes us laugh. By sharing it, more people can have a friend like
GPT-4 to help them.

3. Helping the World: GPT-4 can do many things, like helping doctors find cures for
sicknesses or helping farmers grow more food. By making it open source, we can use its
power to make the world a better place for everyone.

I hope you consider my ideas, Sam. Let's work together to make GPT-4 a friend for
everyone.

Sincerely,
Freddy
CPU times: user 

In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Kate, the Vice president of USA, you are against regulatory capture and like to explain that!",
         max_length=512)

 Subject: Embracing Transparency: The Benefits of Open Sourcing GPT-4

Dear Sam,

As the Vice President of the United States, I am committed to promoting transparency and
innovation in technology. I believe that open-sourcing GPT-4, the next generation of
advanced language models, will bring numerous advantages to society. Here are some key
reasons to consider:

1. Enhanced Collaboration: By making GPT-4 open source, researchers, developers, and
businesses worldwide can collaborate on its development, leading to faster innovation and
better outcomes. This collaboration will also foster a more diverse range of perspectives,
ultimately improving the model's performance.

2. Greater Accessibility: Open-sourcing GPT-4 will make it more accessible to a broader
range of users, including those without the financial means to access proprietary AI
models. This will enable a more equitable distribution of advanced technology, benefiting
individuals and organizations across various sectors.

3. I

In [ ]:
%%time
generate('What is the capital of England?',
         max_length=256)

 The capital of England is London.
CPU times: user 415 ms, sys: 2.03 ms, total: 417 ms
Wall time: 415 ms


In [ ]:
%%time
generate('Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.',
         max_length=256)

 To determine if it's possible for Geoffrey Hinton to have a conversation with George
Washington, we need to consider the time difference between them and the current state of
technology.

Geoffrey Hinton is a contemporary British-Canadian computer scientist, born in 1947. He is
known for his work in artificial intelligence and deep learning.

George Washington was the first President of the United States, serving from 1789 to 1797.
He was born in 1732 and passed away in 1799.

Given the significant time difference between these two individuals, it is not possible
for them to have a direct conversation in the traditional sense. However, we can imagine a
hypothetical scenario where their ideas and thoughts could be represented through their
respective works or writings.

In this case, we could analyze their writings, speeches, and other works to understand
their perspectives and opinions on various topics. This would allow us to create a sort of
"conversation" by comparing their ideas a

In [ ]:
generate('Write a story about a Koala playing pool and beating all the camelids.',
         max_length=512)

 In the heart of a quaint Australian town, there lived a friendly and talented koala named
Koala-Bear. He was known for his exceptional skills in various activities, from climbing
trees to playing sports. One day, Koala-Bear decided to try his hand at a new game, pool.

As he entered the local pool hall, he noticed a group of camelids gathered around a table,
playing a game of pool. The camelids were a lively bunch, consisting of a llama, an
alpaca, and a camel. They were all quite skilled at the game, and their friendly rivalry
had created a lively atmosphere in the room.

Koala-Bear, being the curious and adventurous creature he was, decided to join them. He
approached the table and politely asked if he could play a game with them. The camelids,
always eager for a challenge, agreed to let him join their game.

As the game began, Koala-Bear quickly realized that he had a natural talent for pool. His
long arms and flexible fingers allowed him to maneuver the cue stick with ease, and hi

## Chat

In [ ]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         max_length=128)

 Sure, try these tips to improve your focus while studying:

1. Create a dedicated study space: Designate a specific area for studying, free from
distractions. Make sure it's comfortable and well-lit.

2. Set a schedule: Plan your study sessions in advance, and stick to them. This will help
you manage your time effectively and avoid procrastination.

3. Take breaks: Short breaks can help refresh your mind and improve focus. Take a 5-10
minute break every hour or so to stretch, walk around, or do something relaxing.

4.


In [ ]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         max_length=128)

 Sure, try these tips to improve your focus while studying:

1. Create a dedicated study space: Designate a specific area for studying, free from
distractions. Make sure it's comfortable and well-lit.

2. Set a schedule: Plan your study sessions in advance, and stick to them. This will help
you manage your time effectively and avoid procrastination.

3. Take breaks: Short breaks can help refresh your mind and improve focus. Take a 5-10
minute break every hour or so to stretch, walk around, or do something relaxing.

4.


## GSM8K

In [ ]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         max_length=256)

 Step 1: Determine the number of apples left after lunch. The cafeteria initially had 23
apples and used 20 for lunch. So, 23 - 20 = 3 apples were left.

Step 2: Add the 6 more apples that were bought. The cafeteria now has 3 apples from the
initial amount plus the 6 new apples, which is 3 + 6 = 9 apples.

Step 3: Combine the remaining apples from lunch and the new apples bought. The cafeteria
now has 9 apples from the initial amount and the 6 new apples, which is 9 + 6 = 15 apples
in total.

So, the cafeteria has 15 apples after using 20 for lunch and buying 6 more.


In [ ]:
generate("Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         max_length=128)

 First, we need to convert the 50 minutes of babysitting to hours. There are 60 minutes in
an hour, so 50 minutes is equal to 50/60 = 0.8333333333333333 hours. Now, we can calculate
her earnings by multiplying her hourly wage by the time spent babysitting: $12 per hour *
0.8333333333333333 hours = $9.999999999999


In [ ]:
generate("A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?",
         max_length=256)

 In the first hundred years, the monster consumed 847 people in total. Since ships were
built smaller back then, we can assume that the first ship had the same number of people
as the monster consumed in that time period. So, the number of people on the ship the
monster ate in the first hundred years is 847.


## CodeGen

In [ ]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder")

 def is_prime(number):
    if number <= 1:
        return False
    for i in range(2, number):
        if number % i == 0:
            return False
    return True

def print_prime(n):
    for i in range(2, n + 1):
        if is_prime(i):
            print(i)

print_prime(10)
```

Output:
2
3
5
7
```


In [ ]:
generate('''```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """''')

 def detect_prime(n):
    if n <= 1:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True
